<a href="https://colab.research.google.com/github/X4D0/RecSys_CBF/blob/main/Task_CBF_RecSys_1301184233.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#TASK: Please Implement the Recommender Systems Using The Songs dataset (created by all class members)

1. Input: Song title (_st = "Is It The Answer?"), number of recomended songs (_nt = 10 )
2. Process: calculate cosine similarity over TFIDF columns
3. Returning: _nt songs which close to _st based on cosine similarity
4. Powerpoint yang menjelaskan tugas TF/IDF dengan Cosine similarity

Dataset Lagu dapat diisi dan dilihat pada: https://docs.google.com/spreadsheets/d/1vjszULKCcS4LPup3VJ9MofYPiYhcaoXTC4zdohLFwpQ/edit?usp=sharing




Nama : Rayhan Rahmanda<br>
NIM : 1301184233<br>
Kelas : IF-42-GAB02

## Read the data

In [1]:
import pandas as pd
import numpy as np

df = pd.read_excel('https://docs.google.com/spreadsheets/d/' + 
                   '1Iv4DQM8zF7xehUAPgp7lPZRW2SZ88FEQV6PWPvqPxYc' +
                   '/export?gid=0&format=xlsx')
df.head()

,NIM,Submisike,Artis,Judul,Lirik
0,1301180229,1,Reality Club,Is It The Answer?,I make you break\nYou move I take\nLove is the...
1,1301180229,2,Simple Plan,Jet lag,"Whoa, oh, oh\nWhoa, oh, oh\nSo jet-lagged\n\nW..."
2,1301180229,3,The Script,Superheroes,All the life she has seen\nAll the meaner side...
3,1301180229,4,The Script,Breakeven,I'm still alive but I'm barely breathing\nJust...
4,1301180229,5,Green Day,21 Guns,"Do you know what's worth fighting for,\nWhen i..."


## Pre-Processing Text

### Lowercase, Remove Punctuation, Remove "\n"

In [2]:
import re
import string 

# All words to lowercase
df['Lirik'] = df['Lirik'].str.lower()

# Remove tab, new line, ans back slice
df['Lirik'] = [row.replace("\n"," ") for row in df['Lirik']]

# Remove Punctuation
df['Lirik'] = [row.translate(str.maketrans("","",string.punctuation)) for row in df['Lirik']]

# Remove Numbers
df['Lirik'] = [re.sub(r'\d+', '', row) for row in df['Lirik']]

df.head()

,NIM,Submisike,Artis,Judul,Lirik
0,1301180229,1,Reality Club,Is It The Answer?,i make you break you move i take love is the a...
1,1301180229,2,Simple Plan,Jet lag,whoa oh oh whoa oh oh so jetlagged what time ...
2,1301180229,3,The Script,Superheroes,all the life she has seen all the meaner side ...
3,1301180229,4,The Script,Breakeven,im still alive but im barely breathing just pr...
4,1301180229,5,Green Day,21 Guns,do you know whats worth fighting for when its ...


### Tokenizing

In [3]:
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize

# Tokenize the Lyrics
df['Tokens'] = df['Lirik'].apply(word_tokenize)

df.head()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


,NIM,Submisike,Artis,Judul,Lirik,Tokens
0,1301180229,1,Reality Club,Is It The Answer?,i make you break you move i take love is the a...,"[i, make, you, break, you, move, i, take, love..."
1,1301180229,2,Simple Plan,Jet lag,whoa oh oh whoa oh oh so jetlagged what time ...,"[whoa, oh, oh, whoa, oh, oh, so, jetlagged, wh..."
2,1301180229,3,The Script,Superheroes,all the life she has seen all the meaner side ...,"[all, the, life, she, has, seen, all, the, mea..."
3,1301180229,4,The Script,Breakeven,im still alive but im barely breathing just pr...,"[im, still, alive, but, im, barely, breathing,..."
4,1301180229,5,Green Day,21 Guns,do you know whats worth fighting for when its ...,"[do, you, know, whats, worth, fighting, for, w..."


### Remove Stopwords

In [4]:
# Removing Stopwords & Punctuation
nltk.download('stopwords')
from nltk.corpus import stopwords

filtered_words = []
for row in df['Tokens']:
    filtered_words.append([
        word.lower() for word in row
        if word.lower() not in nltk.corpus.stopwords.words('english')
    ])

df['Tokens'] = filtered_words
df.head()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


,NIM,Submisike,Artis,Judul,Lirik,Tokens
0,1301180229,1,Reality Club,Is It The Answer?,i make you break you move i take love is the a...,"[make, break, move, take, love, answer, say, w..."
1,1301180229,2,Simple Plan,Jet lag,whoa oh oh whoa oh oh so jetlagged what time ...,"[whoa, oh, oh, whoa, oh, oh, jetlagged, time, ..."
2,1301180229,3,The Script,Superheroes,all the life she has seen all the meaner side ...,"[life, seen, meaner, side, took, away, prophet..."
3,1301180229,4,The Script,Breakeven,im still alive but im barely breathing just pr...,"[im, still, alive, im, barely, breathing, pray..."
4,1301180229,5,Green Day,21 Guns,do you know whats worth fighting for when its ...,"[know, whats, worth, fighting, worth, dying, t..."


### Lemmatization

In [5]:
# Setting the Lemmatization object
nltk.download("wordnet")
lmtzr = nltk.stem.wordnet.WordNetLemmatizer()

# Looping through the words and appending the lemmatized version to a list
stemmed_words = []
for row in df['Tokens']:
    stemmed_words.append([
        # Verbs
        lmtzr.lemmatize(  
            # Adjectives
            lmtzr.lemmatize(  
                # Nouns
                lmtzr.lemmatize(word.lower()), 'a'), 'v')
        for word in row
        if word.lower() not in nltk.corpus.stopwords.words('english')])

# Adding the list as a column in the data frame
df['Tokens'] = stemmed_words
df.head()

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


,NIM,Submisike,Artis,Judul,Lirik,Tokens
0,1301180229,1,Reality Club,Is It The Answer?,i make you break you move i take love is the a...,"[make, break, move, take, love, answer, say, g..."
1,1301180229,2,Simple Plan,Jet lag,whoa oh oh whoa oh oh so jetlagged what time ...,"[whoa, oh, oh, whoa, oh, oh, jetlagged, time, ..."
2,1301180229,3,The Script,Superheroes,all the life she has seen all the meaner side ...,"[life, see, mean, side, take, away, prophet, d..."
3,1301180229,4,The Script,Breakeven,im still alive but im barely breathing just pr...,"[im, still, alive, im, barely, breathe, prayin..."
4,1301180229,5,Green Day,21 Guns,do you know whats worth fighting for when its ...,"[know, whats, worth, fight, worth, die, take, ..."


In [6]:
stemmed_sentences = []

# Spacing out the words in the reviews for each restaurant
for row in df['Tokens']:
    stemmed_string = ''
    for word in row:
        stemmed_string = stemmed_string + ' ' + word
    stemmed_sentences.append(stemmed_string)
    
df['Tokens'] = stemmed_sentences
df.head()

,NIM,Submisike,Artis,Judul,Lirik,Tokens
0,1301180229,1,Reality Club,Is It The Answer?,i make you break you move i take love is the a...,make break move take love answer say go away ...
1,1301180229,2,Simple Plan,Jet lag,whoa oh oh whoa oh oh so jetlagged what time ...,whoa oh oh whoa oh oh jetlagged time miss any...
2,1301180229,3,The Script,Superheroes,all the life she has seen all the meaner side ...,life see mean side take away prophet dream pr...
3,1301180229,4,The Script,Breakeven,im still alive but im barely breathing just pr...,im still alive im barely breathe prayin god d...
4,1301180229,5,Green Day,21 Guns,do you know whats worth fighting for when its ...,know whats worth fight worth die take breath ...


## TF-IDF and Cosine Similarity

In [7]:
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer

# Set "Judul" as Index
df.set_index('Judul', inplace=True)
# TF-IDF
tf = TfidfVectorizer(analyzer='word', ngram_range=(1, 3), min_df=0, stop_words='english')
tfidf_matrix = tf.fit_transform(df['Tokens'])
# Cosine Similarity
cos_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)
cos_sim

array([[1.        , 0.00535797, 0.00986331, ..., 0.0198553 , 0.00995391,
        0.03753962],
       [0.00535797, 1.        , 0.01002504, ..., 0.00192688, 0.01334401,
        0.00586109],
       [0.00986331, 0.01002504, 1.        , ..., 0.00244596, 0.00510894,
        0.00566061],
       ...,
       [0.0198553 , 0.00192688, 0.00244596, ..., 1.        , 0.00129794,
        0.00807914],
       [0.00995391, 0.01334401, 0.00510894, ..., 0.00129794, 1.        ,
        0.00645579],
       [0.03753962, 0.00586109, 0.00566061, ..., 0.00807914, 0.00645579,
        1.        ]])

## "Judul" as Index

In [8]:
indices = pd.Series(df.index)
indices[:10]

0                  Is It The Answer?
1                            Jet lag
2                        Superheroes
3                          Breakeven
4                            21 Guns
5     Wake me up when September Ends
6    You're Gonna Live Forever in Me
7                              Day 1
8                     Like me better
9                               Maps
Name: Judul, dtype: object

## Recommend the film

In [37]:
def recommendations(songtitle, numberrec, cos_sim = cos_sim):
    '''
    input : one songs title, the cosine matrix and number of recommendations

    output : list of n songs (the title) that similar with the input song
    '''
    recommended_songs = []

    # Cek songs title in dataset
    if (songtitle.casefold() == indices.str.casefold()).any():
      # Mengambil songs title berdasarkan variabel indicies
      idx = indices[indices.str.casefold() == songtitle.casefold()].index[0]
    
      # Membuat series berdasarkan skor kesamaan
      score_series = pd.Series(cos_sim[idx]).sort_values(ascending = False)

      # mengambil index dan dibuat 10 baris rekomendasi terbaik
      numberrec_indexes = list(score_series.iloc[1:numberrec+1].index)
      
      for i in numberrec_indexes:
          recommended_songs.append(list(df.index)[i])
          
      return recommended_songs
    else:
      return "Song's title not found. Please make sure you typed the Title correctly."

In [41]:
recommendations('Breakeven',10)

['Not today',
 'Me And My Broken Heart',
 'Here With Me',
 'Grateful',
 'Hysteria',
 'Say Something',
 'Taken',
 'I Want to Break Free',
 "It's My Life",
 'I Wish']